# Inverse problem for the wave equation using an operator recurrent neural network

We consider the inverse problem to find $a$ in the below wave equation given 
the Neumann-to-Dirichlet map 

$$
\Lambda h = u|_{x = 0},
$$

where $u$ is the solution to the problem 

$$
\begin{cases}
\partial_t^2 u - a(x) \partial_x^2 u = 0, & \text{on $(0,T) \times (0,L)$},
\\
\partial_x u|_{x=0} = h, \quad \partial_x u|_{x=L} = 0,
\\
u|_{t=0} = 0, \quad \partial_t u|_{t=0} = 0.
\end{cases}
$$

Here we consider only a subproblem related to the inverse problem to find $a$.
In Section 2 of 

> Jussi Korpela, Matti Lassas and Lauri Oksanen.
> _Regularization strategy for an inverse problem for a 1 + 1 dimensional wave equation_.
> Inverse Problems 32, 065001, 2016.
> <https://doi.org/10.1088/0266-5611/32/6/065001> 

it was shown that $\Lambda$ determines the following volumes 

$$
V(r) = \int_0^{\chi(r)} \frac{1}{c(x)^2} dx
$$

and that these volumes then determine $a$.
Here $c^2 = a$ and $\chi$ is the inverse function of $\tau$ defined by

$$
\tau(y) = \int_0^y \frac{1}{c(x)} dx.
$$


We consider the subproblem to compute a single volume $V(r_0)$, with fixed $r_0>0$, given $\Lambda$. 
We solve this problem using a neural network, with the network architecture taken from 

> Maarten V. de Hoop, Matti Lassas, Christopher A. Wong. _Deep learning architectures for nonlinear operator functions and nonlinear inverse problems_. [arXiv:1912.11090](https://arxiv.org/abs/1912.11090)

The training data consists of pairs $(\Lambda, V(r_0))$ corresponding to different functions $a$.
Here $\Lambda$ is, of course, discretized, and the details of the discretization are discussed in the notebook describing the generation of the data. 


# Initialization

In [ ]:
import numpy as np
import torch
import os #help navigate files in folder
import matplotlib.pyplot as plt #plotting
import pandas as pd #data frames = nice table

import opnet
from volume_inversion_data import generate_data, save_data, load_data

#finding the best lr
#PATH1 = './volume_inversion_net1.pth'
#PATH2 = './volume_inversion_net2.pth'
#PATH3 = './volume_inversion_net3.pth'
#PATH4 = './volume_inversion_net4.pth'

# teach until limit point
#PATH1 = './volume_inversion_net1limit.pth'
#PATH2 = './volume_inversion_net2limit.pth'
#PATH3 = './volume_inversion_net3limit.pth'
#PATH4 = './volume_inversion_net4limit.pth'

#relu vs no relu
PATH1 = './volume_inversion_net_1no_relu.pth'
PATH2 = './volume_inversion_net_2no_relu.pth' 
PATH3 = './volume_inversion_net_3no_relu.pth' 
PATH4 = './volume_inversion_net_4no_relu.pth' 
PATH5 = './volume_inversion_net_1relu.pth' 
PATH6 = './volume_inversion_net_2relu.pth' 
PATH7 = './volume_inversion_net_3relu.pth' 
PATH8 = './volume_inversion_net_4relu.pth'

#PATH = './volume_inversion_net.pth' #network that can be overwritten
#PATH = './volume_inversion_ReLU3.pth' #biggest data (10000/2500)
#PATH = './volume_inversion_NOReLU3.pth' #biggest data (10000/2500)
#PATH = './volume_inversion_NOReLU2.pth' #big data (6000/1000)
#PATH = './volume_inversion_ReLU2.pth' #big data (6000/1000)
#PATH = './volume_inversion_NOReLU.pth' #normal data (600/100)
#PATH = './volume_inversion_ReLU.pth' #normal data (600/100)
#PATH = './volume_inversion_netLONG.pth' # trained 1600 epochs

# Generation of training data

In [ ]:
#save_data(*generate_data(30000), "volume_inversion_train_data2.npz")
#save_data(*generate_data(5000), "volume_inversion_test_data2.npz")

train_data_path = "volume_inversion_train_dataBIG.npz"
test_data_path = "volume_inversion_test_dataBIG.npz"

# Training

In [ ]:
# Specify the dimension and the loss function
dim = 126 # this needs be the size of Lambda_h
num_layers = 5
loss_fn = torch.nn.MSELoss()

import wave_training_and_testing
import os.path
import time

# update changes
from importlib import reload 
reload(wave_training_and_testing)
reload(opnet)

lr=1e-4
losses_list=[]
path_list = [PATH1, PATH2, PATH3, PATH4, PATH5, PATH6, PATH7, PATH8]
for path in path_list:
    start_time=time.perf_counter()
    
    if path in [PATH1, PATH2, PATH3, PATH4]: #network without relu
        model = opnet.OperatorNet(dim, 2*num_layers, scalar_output=True, useReLU=False)
        losses_list.append(wave_training_and_testing.wave_training_and_testing(model, loss_fn, lr, path, train_data_path, test_data_path))
    else: #network with relu
        model = opnet.OperatorNet(dim, num_layers, scalar_output=True, useReLU=True)
        losses_list.append(wave_training_and_testing.wave_training_and_testing(model, loss_fn, lr, path, train_data_path, test_data_path))

    #create the new network
    #model = opnet.OperatorNet(dim, 2*num_layers, scalar_output=True, useReLU=False)
    #model = opnet.OperatorNet(dim, num_layers, scalar_output=True, useReLU=True)
    
    #teach the network and save average losses to a list in losses_list:
    #losses_list.append(wave_training_and_testing.wave_training_and_testing(model, loss_fn, lr, path, train_data_path, test_data_path))
    
    end_time = time.perf_counter()
    print(f"This took {(end_time - start_time)/60:0.4} minutes \n")

In [ ]:
#Plotting
#Livas code + own modification
from matplotlib.lines import lineStyles
#plt.plot(2*np.arange(1, len(losses_list[0])+1),losses_list[0], label = 'Network 1', linestyle='--')
#plt.plot(2*np.arange(1, len(losses_list[1])+1),losses_list[1], label = 'Network 2', linestyle='--')
#plt.plot(2*np.arange(1, len(losses_list[2])+1),losses_list[2], label = 'Network 3', linestyle='--')
#plt.plot(2*np.arange(1, len(losses_list[3])+1),losses_list[3], label = 'Network 4', linestyle='--')

plt.plot(2*np.arange(1, len(losses_list[0])+1),losses_list[0], label = 'Network without relu 1', linestyle='-')
plt.plot(2*np.arange(1, len(losses_list[1])+1),losses_list[1], label = 'Network without relu 2', linestyle='-')
plt.plot(2*np.arange(1, len(losses_list[2])+1),losses_list[2], label = 'Network without relu 3', linestyle='-')
plt.plot(2*np.arange(1, len(losses_list[3])+1),losses_list[3], label = 'Network without relu 4', linestyle='-')
plt.plot(2*np.arange(1, len(losses_list[4])+1),losses_list[4], label = 'Network with relu 1', linestyle='--')
plt.plot(2*np.arange(1, len(losses_list[5])+1),losses_list[5], label = 'Network with relu 2', linestyle='--')
plt.plot(2*np.arange(1, len(losses_list[6])+1),losses_list[6], label = 'Network with relu 3', linestyle='--')
plt.plot(2*np.arange(1, len(losses_list[7])+1),losses_list[7], label = 'Network with relu 4', linestyle='--')

plt.xlabel("Epochs")
plt.ylabel("Average Loss")
plt.grid()
plt.legend()
plt.savefig("../masters-thesis/data/wave-090123-reluvsnorelu.eps", format="eps") #save as eps so it's easy to insert it to latex-file

In [ ]:
#Table
# x-coordinates: 
longest = (max(len(i) for i in losses_list)) # the length of the longest list
epo = np.arange(1,longest+1,1) # now from 1 to longest and every number. The interval is half open [1,x) so there has to be +1
index_x = [f"{2*e} epochs" for e in epo]
# y-coordinates: 
index_y = [1,2,3,4]
df1 = pd.DataFrame(losses_list, columns=index_x, index = index_y)
df1.index.name = "Network"
# saves the table in latex-code
df1.to_latex("../masters-thesis/data/wave-090123-reluvsnorelu.tex", columns=[f"{e} epochs" for e in [10, 50, 100, 150, 200]])
# saves the table in csv-form
df1.to_csv("../masters-thesis/data/wave-090123-reluvsnorelu.csv", index=True, sep=";", decimal=",") #sep=separates columns, decimal=uses , instead of .
df1